# Projet Final Apache Spark

**Nom Etudiant :**  SOME

**Prenom Etudiant:**  Tiare Hermann Williams

**Classe :**  M1 BDA


## Description
Ce projet consiste à utiliser Apache Spark pour faire l'analyse et le traitement des données de **[San Francisco Fire Department Calls ](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)** afin de fournir quelques KPI (*Key Performance Indicator*). Le **SF Fire Dataset** comprend les réponses aux appels de toutes les unités d'incendie. Chaque enregistrement comprend le numéro d'appel, le numéro d'incident, l'adresse, l'identifiant de l'unité, le type d'appel et la disposition. Tous les intervalles de temps pertinents sont également inclus. Étant donné que ce Dataset est basé sur les réponses et que la plupart des appels impliquent plusieurs unités, ainsi il existe plusieurs enregistrements pour chaque numéro d'appel. Les adresses sont associées à un numéro de bloc, à une intersection ou à une boîte d'appel, et non à une adresse spécifique.

**Plus de details sur la description des données cliquer sur ce [lien](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)**

## Travail à faire.
L'objectif de ce projet est de comprendre le **SF Fire Dataset** afin de bien répondre aux questions en utilisant les codes Spark/Scala adéquates.

- Créer un repos git (public ou privé) et partager le repos avec mon mail (limahin10@gmail.com)
- Ecrire un code lisible et bien indenté 
- N'oublier pas de mettre en commentaire la justification de vos réponses sur les cellules Markdown. 


## Note:
- Le projet est personnel, c'est-à-dire chaque notebook ne concerne qu'un seul étudiant. 
- Deadline : **Jeudi 10 janvier 2021** (Aucune de dérogation ne sera acceptée)

### Chargement des données

Importation des packages Spark

In [2]:
import org.apache.spark.sql.types._ 
import org.apache.spark.sql.functions._ 
import spark.implicits._
import org.apache.spark.sql.Column
import org.apache.spark.sql.SQLContext

import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._
import org.apache.spark.sql.Column
import org.apache.spark.sql.SQLContext


Nous allons jeter un coup d'oeil sur la structure des données avant de définir un schéma

In [3]:
!head -1 "datasets/sf-fire/sf-fire-calls.csv"

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay



Vu que la taille de ces données est énormes, inferer le schema pour un très grande volumes de données s'avère un peu couteux. Nous allons ainsi définir un schema pour le Dataset.

In [4]:
val fireSchema = StructType(Array(StructField("CallNumber", IntegerType, true),
  StructField("UnitID", StringType, true),
  StructField("IncidentNumber", IntegerType, true),
  StructField("CallType", StringType, true),                  
  StructField("CallDate", StringType, true),      
  StructField("WatchDate", StringType, true),
  StructField("CallFinalDisposition", StringType, true),
  StructField("AvailableDtTm", StringType, true),
  StructField("Address", StringType, true),       
  StructField("City", StringType, true),       
  StructField("Zipcode", IntegerType, true),       
  StructField("Battalion", StringType, true),                 
  StructField("StationArea", StringType, true),       
  StructField("Box", StringType, true),       
  StructField("OriginalPriority", StringType, true),       
  StructField("Priority", StringType, true),       
  StructField("FinalPriority", IntegerType, true),       
  StructField("ALSUnit", BooleanType, true),       
  StructField("CallTypeGroup", StringType, true),
  StructField("NumAlarms", IntegerType, true),
  StructField("UnitType", StringType, true),
  StructField("UnitSequenceInCallDispatch", IntegerType, true),
  StructField("FirePreventionDistrict", StringType, true),
  StructField("SupervisorDistrict", StringType, true),
  StructField("Neighborhood", StringType, true),
  StructField("Location", StringType, true),
  StructField("RowID", StringType, true),
  StructField("Delay", FloatType, true)))

fireSchema: org.apache.spark.sql.types.StructType = StructType(StructField(CallNumber,IntegerType,true), StructField(UnitID,StringType,true), StructField(IncidentNumber,IntegerType,true), StructField(CallType,StringType,true), StructField(CallDate,StringType,true), StructField(WatchDate,StringType,true), StructField(CallFinalDisposition,StringType,true), StructField(AvailableDtTm,StringType,true), StructField(Address,StringType,true), StructField(City,StringType,true), StructField(Zipcode,IntegerType,true), StructField(Battalion,StringType,true), StructField(StationArea,StringType,true), StructField(Box,StringType,true), StructField(OriginalPriority,StringType,true), StructField(Priority,StringType,true), StructField(FinalPriority,IntegerType,true), StructField(ALSUnit,BooleanType,true)...


In [5]:
val sfFireFile = "datasets/sf-fire/sf-fire-calls.csv"
val fireDF = spark
  .read
  .schema(fireSchema)
  .option("header", "true")
  .csv(sfFireFile)

sfFireFile: String = datasets/sf-fire/sf-fire-calls.csv
fireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


Nous allons mettre en cache le Dataframe

In [6]:
fireDF.cache()

res1: fireDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


In [6]:
fireDF.count()

res1: Long = 175296


In [7]:
fireDF.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [8]:
fireDF.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

Filtrage des d'appels de type "Medical Incident"

In [7]:
val fewFireDF = fireDF
  .select("IncidentNumber", "AvailableDtTm", "CallType") 
  .where($"CallType" =!= "Medical Incident")

fewFireDF.show(5, false)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



fewFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentNumber: int, AvailableDtTm: string ... 1 more field]


### Question 1
**Combien de types d'appels distincts ont été passés ?**  
Pour être sûr, il ne faut pas compter les valeurs «nulles» dans la colonne.

In [8]:
// Reponse 1
fireDF.select("CallType").distinct().count()

res3: Long = 30


En fonction de la variable CallType nous denombrons 30 types d'appels distinct dans la base de donnees. Le nombre d'appels par type sera commenté a la question 4. La base de comporte pas de valeurs manquantes apparemments.

### Question 2

**Quels types d'appels différents ont été passés au service d'incendie?**

In [9]:
// Reponse 2
fireDF.select("CallType").distinct().show(10, false)

+-----------------------------------+
|CallType                           |
+-----------------------------------+
|Elevator / Escalator Rescue        |
|Marine Fire                        |
|Aircraft Emergency                 |
|Confined Space / Structure Collapse|
|Administrative                     |
|Alarms                             |
|Odor (Strange / Unknown)           |
|Citizen Assist / Service Call      |
|HazMat                             |
|Watercraft in Distress             |
+-----------------------------------+
only showing top 10 rows



Nous presentons ici les 10 premieres lignes des differents type d'appels

### Question 3

**Trouver toutes les réponses ou les délais sont supérieurs à 5 minutes?**

*Indication
1. Renommer la colonne Delay -> ReponseDelayedinMins
2. Retourner un nouveau DataFrame
3. Afficher tous les appels où le temps de réponse à un site d'incendie a eu lieu après un retard de plus de 5 minutes

In [10]:
// Reponse 3
val newFireDF = fireDF.withColumnRenamed("Delay", "ResponseDelayedinMins")
val fireCallssup5 = newFireDF.filter(newFireDF("ResponseDelayedinMins")>5)
fireCallssup5.show(10, false)

+----------+------+--------------+-----------------------------+----------+----------+--------------------+----------------------+------------------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------------+--------------------------+----------------------+------------------+------------------------------+-------------------------------------+-------------+---------------------+
|CallNumber|UnitID|IncidentNumber|CallType                     |CallDate  |WatchDate |CallFinalDisposition|AvailableDtTm         |Address                       |City|Zipcode|Battalion|StationArea|Box |OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType      |UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|Neighborhood                  |Location                             |RowID        |ResponseDelayedinMins|
+----------+------+--------------+-----------------------------+----------

newFireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
fireCallssup5: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallNumber: int, UnitID: string ... 26 more fields]


### Transformations des dates
Maintenant nous allons d'abord:
1. Transformer les dates de type String en Spark Timestamp afin que nous puissions effectuer des requêtes basées sur la date plus tard
2. Retourner le Dataframe transformée
3. Mettre en cache le nouveau DataFrame

In [11]:
val fireTSDF = newFireDF
  .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
  .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate") 
  .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm")

fireTSDF.cache()

fireTSDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
res6: fireTSDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


In [69]:
fireTSDF.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 |-- SupervisorDistrict: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Location: string (nullable =

### Question 4
**Quels sont les types d'appels les plus courants?**

In [12]:
//Reponse 4
val fireCallsCourant = fireTSDF.groupBy("CallType").count().orderBy(desc("count"))
fireCallsCourant.show(30, false)

+--------------------------------------------+------+
|CallType                                    |count |
+--------------------------------------------+------+
|Medical Incident                            |113794|
|Structure Fire                              |23319 |
|Alarms                                      |19406 |
|Traffic Collision                           |7013  |
|Citizen Assist / Service Call               |2524  |
|Other                                       |2166  |
|Outside Fire                                |2094  |
|Vehicle Fire                                |854   |
|Gas Leak (Natural and LP Gases)             |764   |
|Water Rescue                                |755   |
|Odor (Strange / Unknown)                    |490   |
|Electrical Hazard                           |482   |
|Elevator / Escalator Rescue                 |453   |
|Smoke Investigation (Outside)               |391   |
|Fuel Spill                                  |193   |
|HazMat                     

fireCallsCourant: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallType: string, count: bigint]


Les Incidents medicaux representent les types d'appels courant dans la base de donnees avec 11794 appels celui pour les besoin administrative est le moins frequent

### Question 5-a
**Quels sont boites postaux rencontrés dans les appels les plus courants?**

In [13]:
//Reponse
val fireTSDFBox = fireTSDF.select("IncidentNumber", "Box", "CallType").where($"CallType" === "Medical Incident")

fireTSDFBox.show()

+--------------+----+----------------+
|IncidentNumber| Box|        CallType|
+--------------+----+----------------+
|       2003241|6495|Medical Incident|
|       2003242|1455|Medical Incident|
|       2003343|3513|Medical Incident|
|       2003348|5415|Medical Incident|
|       2003381|5525|Medical Incident|
|       2003399|1557|Medical Incident|
|       2003403|7173|Medical Incident|
|       2003409|8635|Medical Incident|
|       2003417|7145|Medical Incident|
|       2003417|7145|Medical Incident|
|       2003435|1311|Medical Incident|
|       2003500|1153|Medical Incident|
|       2003529|8734|Medical Incident|
|       2003550|6244|Medical Incident|
|       2003576|7113|Medical Incident|
|       2003577|4252|Medical Incident|
|       2003584|5226|Medical Incident|
|       2003593|4256|Medical Incident|
|       2003630|3361|Medical Incident|
|       2003639|5246|Medical Incident|
+--------------+----+----------------+
only showing top 20 rows



fireTSDFBox: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentNumber: int, Box: string ... 1 more field]


Nous nous sommes focalisé sur les types d'appel le plus frequnet c'est a dire : Medical Incident. et nous affichons ici les 20 premiere boites postaux

### Question 5-a
**Quels sont les quartiers de San Francisco dont les codes postaux sont 94102 et 94103?**

In [14]:
//Reponse 5-b
val fireTSDFNeigh = fireTSDF.select("Neighborhood").where($"Box" === "94102" || $"Box" === "94103").distinct()
fireTSDFNeigh.show(5, false)

+------------+
|Neighborhood|
+------------+
+------------+



fireTSDFNeigh: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Neighborhood: string]


aucun quartier de San Francisco n'a comme code postal 94102 et 94103

### Question 6
**Determiner le nombre total d'appels, ainsi que la moyenne, le minimum et le maximum du temps de réponse des appels?**

In [15]:
//Reponse 6
val desc = fireTSDF.describe()
desc.select("summary", "ResponseDelayedinMins").show()

+-------+---------------------+
|summary|ResponseDelayedinMins|
+-------+---------------------+
|  count|               175296|
|   mean|    3.892364154521585|
| stddev|    9.378286226254216|
|    min|          0.016666668|
|    max|              1844.55|
+-------+---------------------+



desc: org.apache.spark.sql.DataFrame = [summary: string, CallNumber: string ... 23 more fields]


En moyenne les appels ne dure que 4 mn

### Question 7-a
**Combien d'années distinctes trouve t-on dans ce Dataset?**  
Dans ce dataset nous avons des données comprises entre 2000-2018. Vous pouvez utilisez la fonction Spark `year()` pour les dates en Timestamp

In [16]:
//Reponse 7-a
fireTSDF.select(year(fireTSDF("IncidentDate"))).distinct().orderBy("year(IncidentDate)").show()

+------------------+
|year(IncidentDate)|
+------------------+
|              2000|
|              2001|
|              2002|
|              2003|
|              2004|
|              2005|
|              2006|
|              2007|
|              2008|
|              2009|
|              2010|
|              2011|
|              2012|
|              2013|
|              2014|
|              2015|
|              2016|
|              2017|
|              2018|
+------------------+



### Question 7-b
**Quelle semaine de l'année 2018 a eu le plus d'appels d'incendie?**

In [17]:
//Reponse 7-b
val df_week = fireTSDF.filter(year(fireTSDF("IncidentDate")) === "2018").groupBy(weekofyear(fireTSDF("IncidentDate"))).count().orderBy("count")
df_week.show(52, false)

+------------------------+-----+
|weekofyear(IncidentDate)|count|
+------------------------+-----+
|45                      |64   |
|24                      |198  |
|4                       |202  |
|17                      |203  |
|36                      |203  |
|30                      |203  |
|41                      |220  |
|12                      |221  |
|35                      |221  |
|15                      |222  |
|37                      |223  |
|26                      |223  |
|27                      |223  |
|29                      |223  |
|3                       |224  |
|39                      |224  |
|20                      |225  |
|6                       |225  |
|14                      |225  |
|33                      |225  |
|38                      |226  |
|9                       |228  |
|7                       |228  |
|16                      |228  |
|28                      |231  |
|21                      |231  |
|34                      |232  |
|10       

df_week: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [weekofyear(IncidentDate): int, count: bigint]


la semaine 22 represente la semaine avec le plus d'appels

### Question 8
**Quels sont les quartiers de San Francisco qui ont connu le pire temps de réponse en 2018?**

In [18]:
//Reponse 8
val TSDF = fireTSDF.filter(year(fireTSDF("IncidentDate")) === "2018")
val stat = TSDF.groupBy("Neighborhood").mean("ResponseDelayedinMins").orderBy("avg(ResponseDelayedinMins)")
stat.show(100,false)

+------------------------------+--------------------------+
|Neighborhood                  |avg(ResponseDelayedinMins)|
+------------------------------+--------------------------+
|Lone Mountain/USF             |3.2472222397724786        |
|Glen Park                     |3.277777776122093         |
|Western Addition              |3.2843276457861066        |
|Lincoln Park                  |3.3111111190583973        |
|Portola                       |3.3385542175137854        |
|Japantown                     |3.347695054526025         |
|None                          |3.363333320617676         |
|Hayes Valley                  |3.3703894590082037        |
|Marina                        |3.4267888378098372        |
|Mission                       |3.4505293849459875        |
|Visitacion Valley             |3.4838095208009086        |
|Noe Valley                    |3.525949378556843         |
|Sunset/Parkside               |3.5842003932147355        |
|Outer Mission                 |3.631751

TSDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallNumber: int, UnitID: string ... 26 more fields]
stat: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Neighborhood: string, avg(ResponseDelayedinMins): double]


le quartier avec le pire temps de reponde supposé ici la moyenne du temps de reponse est Chinatown

### Question 9

**Comment stocker les données du Dataframe sous format de fichiers Parquet?**

In [21]:
//Reponse 9
fireTSDF.write.format("parquet").save("/tmp/fireServiceParquet1/")

### Question 10
**Comment relire les données stockée en format Parquet?**

In [22]:
//Reponse 10
val tempDF = spark.read.parquet("/tmp/fireServiceParquet1/")
tempDF.show(5, false)

+----------+------+--------------+----------------+--------------------+-----------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+---------------------+-------------------------------------+-------------+---------------------+-------------------+-------------------+-------------------+
|CallNumber|UnitID|IncidentNumber|CallType        |CallFinalDisposition|Address                |City|Zipcode|Battalion|StationArea|Box |OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|Neighborhood         |Location                             |RowID        |ResponseDelayedinMins|IncidentDate       |OnWatchDate        |AvailableDtTS      |
+----------+------+--------------+----------------+--------------------+-----------------------+----+-------+-----

tempDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


## FIN